In [1]:
import pandas as pd
import vaex as vx
import numpy as np
from datemerge import datemerge
from hyperlink import hyperlink
from to_excel_file import to_excel_file

In [2]:
df = pd.read_excel(r"D:\\Data Kompa\\Hotpot\\Tính Hotpot.xlsx", sheet_name='Data')

In [3]:
sentiment = ['Positive','Neutral','Negative']
main = ['Ashima','KTOP','Mala Gang']
compe = ['Asanoha','Momo Paradise','Dookki','Mala Panda','Yangguofu']
all_brands = main + compe
channel = ['Facebook','Fanpage','Food Review','Forum','News','Tiktok','Youtube','Google Review','Instagram','Threads','Linkedin','E-commerce']

In [4]:
Link = r"D:\\Data Kompa\\Hotpot\\OutputQ4.xlsx"

In [10]:
#SHEET sentiment_interaction

#sentiment
df[['Sentiment','Labels1','Labels2','Likes','Shares','Comments']] = df[['Sentiment','Labels1','Labels2','Likes','Shares','Comments']].fillna(0)

SOV = df.pivot_table(index='Topic',columns='Sentiment',values='Id',aggfunc='count').fillna(0).reset_index()
SOV.columns = [col[1] if isinstance(col, tuple) else col for col in SOV.columns]
SOV.rename(columns={0: 'Khác'},inplace=True)
SOV['Total'] = SOV[['Positive', 'Neutral', 'Negative', 'Khác']].sum(axis=1)
SOV = SOV[['Topic', 'Positive', 'Neutral', 'Negative', 'Khác', 'Total']]
df['Topic'] = pd.Categorical(df['Topic'], categories=all_brands, ordered=True)
df = df.sort_values('Topic').reset_index(drop=True)
print(SOV)

#Interaction
df['Topic'] = df['Topic'].str.strip()
df_main = df[df['Topic'].isin(main)].reset_index()
inter = df_main.pivot_table(index='Topic', values=['Comments', 'Shares', 'Likes'], aggfunc='sum').fillna(0).reset_index()
inter = inter[['Topic','Comments','Shares','Likes']]


count_post = df_main.pivot_table(index='Topic',columns='post',values='Id',aggfunc='count').fillna(0).reset_index()
count_post

author = df_main.drop_duplicates(subset=['Topic', 'AuthorId']).pivot_table(index='Topic',values='AuthorId',aggfunc='count').reset_index()
author

interaction = pd.merge(pd.merge(count_post, inter, on='Topic'), author, on='Topic')
print('\n',interaction)

           Topic  Positive  Neutral  Negative     Khác     Total
0        Asanoha     479.0    603.0       3.0    267.0    1352.0
1         Ashima      49.0   3349.0       3.0   1797.0    5198.0
2         Dookki    4392.0  52731.0     609.0  51191.0  108923.0
3           KTOP       5.0    761.0       4.0      0.0     770.0
4      Mala Gang     304.0   1568.0      16.0    517.0    2405.0
5     Mala Panda      55.0    303.0       2.0      0.0     360.0
6  Momo Paradise       0.0     34.0       0.0      0.0      34.0
7      Yangguofu       0.0     37.0       0.0      0.0      37.0

        Topic  post MXH  post News  Comments   Shares    Likes  AuthorId
0     Ashima    4823.0       58.0    3724.0   1160.0  58792.0       623
1       KTOP     748.0        0.0    1108.0     60.0   1332.0        77
2  Mala Gang    1242.0        0.0    2487.0  11615.0  60743.0       879


In [11]:
with pd.ExcelWriter(Link, engine='openpyxl') as writer:
    SOV.to_excel(writer, sheet_name='Sentiment_Interaction', startrow=0, startcol=0, index=False)
    interaction.to_excel(writer, sheet_name='Sentiment_Interaction', startrow=0, startcol=9, index=False)

In [12]:
#SHEET sentiment.channel
sent_chn = df.pivot_table(index=['Topic','Sentiment','POE'],columns=['Channel'],values='Id',aggfunc='count').reset_index().reindex(columns = ['Topic','Sentiment','POE'] + channel).fillna(0)
sent_chn = sent_chn[sent_chn['Sentiment'] != 0]

total_poe = df.pivot_table(index=['Topic','Sentiment','POE'],values='Id',aggfunc='count').reset_index()
total_setiment_topic = df.pivot_table(index=['Topic','Sentiment'],values='Id',aggfunc='count').reset_index()
total_poe = pd.merge(total_poe,total_setiment_topic,on=['Topic','Sentiment'])
total_poe['per_POE_by_sent'] = total_poe['Id_x']/total_poe['Id_y'] 
total_poe = total_poe.drop(columns=['Id_x','Id_y'])

sent_chn = pd.merge(sent_chn,total_poe,on=['Topic','Sentiment','POE'])

sent_chn

,Topic,Sentiment,POE,Facebook,Fanpage,Food Review,Forum,News,Tiktok,Youtube,Google Review,Instagram,Threads,Linkedin,E-commerce,per_POE_by_sent
0,Asanoha,Negative,Earned,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667
1,Asanoha,Negative,Owned,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333
2,Asanoha,Neutral,Earned,43.0,0.0,0.0,0.0,3.0,445.0,0.0,0.0,1.0,0.0,0.0,0.0,0.815920
3,Asanoha,Neutral,Owned,0.0,97.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.184080
4,Asanoha,Positive,Earned,1.0,0.0,0.0,0.0,0.0,470.0,0.0,0.0,0.0,0.0,0.0,0.0,0.983299
5,Asanoha,Positive,Owned,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.016701
6,Ashima,Negative,Earned,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.000000
7,Ashima,Neutral,Earned,2887.0,0.0,10.0,0.0,41.0,17.0,1.0,0.0,0.0,2.0,0.0,0.0,0.883249
8,Ashima,Neutral,Owned,0.0,273.0,0.0,0.0,0.0,40.0,0.0,0.0,37.0,0.0,0.0,0.0,0.104509
9,Ashima,Neutral,Paid,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012242


In [13]:
#SHEET sentiment_labels
Labels = df.pivot_table(index=['Topic','Labels1','Labels2'],columns='Sentiment',values='Id',aggfunc='count').fillna(0).reset_index()
Labels = Labels[Labels['Labels1'] != 0]
Labels = Labels[['Topic','Labels1','Labels2','Positive','Neutral','Negative']]
Labels

Sentiment,Topic,Labels1,Labels2,Positive,Neutral,Negative
1,Asanoha,Chương trình MKT,Bài đăng quảng bá,100.0,422.0,1.0
2,Asanoha,Chương trình MKT,Hoạt động quảng bá nhà hàng,0.0,45.0,1.0
3,Asanoha,Chương trình MKT,Khai trương chi nhánh mới,1.0,0.0,0.0
4,Asanoha,Cơ sở vật chất,Hình ảnh không gian,20.0,0.0,0.0
5,Asanoha,Dịch vụ,Phục vụ,1.0,0.0,0.0
...,...,...,...,...,...,...
98,Momo Paradise,Chương trình MKT,Hoạt động quảng bá nhà hàng,0.0,33.0,0.0
99,Yangguofu,Chương trình MKT,Bài đăng quảng bá,0.0,1.0,0.0
100,Yangguofu,Chương trình MKT,Hoạt động quảng bá nhà hàng,0.0,33.0,0.0
101,Yangguofu,Chương trình MKT,Khai trương chi nhánh mới,0.0,1.0,0.0


In [14]:
with pd.ExcelWriter(Link, engine='openpyxl', mode='a') as writer:
    sent_chn.to_excel(writer, sheet_name='Sentiment.Channel', startrow=0, startcol=0, index=False)
    Labels.to_excel(writer, sheet_name='Sentiment.Labels', startrow=0, startcol=0, index=False)

REPORT

mainbrand

In [4]:
period = 'Năm'
period_time = "Năm 2024"

In [5]:
sent = pd.read_excel(r"D:\\Data Kompa\\Hotpot\\Tính Hotpot.xlsx", sheet_name='Sentiment.Interaction')
chn = pd.read_excel(r"D:\\Data Kompa\\Hotpot\\Tính Hotpot.xlsx", sheet_name='Sentiment.Channel')
lb = pd.read_excel(r"D:\\Data Kompa\\Hotpot\\Tính Hotpot.xlsx", sheet_name='Sentiment.Labels')
#demo = pd.read_excel(r"D:\\Data Kompa\\Hotpot\\HOTPOTquar2.xlsx", sheet_name='Demo')

In [6]:
slide_n_dict = {}

In [7]:
#chn_main = chn[(chn['Quý'] == quar)]

df_list=[]
for i in main:
    df_chn = chn[(chn[period] == period_time) & (chn['Brand']==i)]
    overall = df_chn.groupby('Brand')['MXH', 'News', 'Total final'].sum().reset_index()
    df_list.append(overall)
    
    
    total_overall = df_chn[['MXH', 'News', 'Total final']].sum()
    sent_chn_o =(
    df_chn.groupby(['Sentiment'])[['MXH', 'News', 'Total final']]
    .sum()
    .div(total_overall)
    .fillna(0)
    .round(3)
    .reindex(sentiment)
    .reset_index())
    df_list.append(sent_chn_o)
    
    
    df_working = df[df['Topic']==i]
    date = datemerge(df_working.groupby(['PublishedDate']).agg({'Id': 'count'}).pivot_table(index='PublishedDate', values='Id', aggfunc='sum'), '2024-01-01', '2024-12-31')
    df_list.append(date)
    
    sent_total = df_chn.groupby(['Sentiment'])['Total final'].sum().round(0).reindex(sentiment).reset_index()
    df_list.append(sent_total)
    
    df_lb = lb[(lb[period] == period_time) & (lb['Brand']==i)]
    labels = df_lb.groupby(['Labels2'])['Brand','Labels2','Total final'].sum().round(0).sort_values(by='Total final',ascending = False)
    df_list.append(labels)
    

    sent_by_chn = df_chn.pivot_table(index=['Sentiment'], values=channel, aggfunc='sum').round(0)
    sent_by_chn = sent_by_chn.transpose().reindex(channel, columns = sentiment)
    sent_by_chn['Total'] = sent_by_chn.sum(axis=1)
    df_list.append(sent_by_chn)
    
    poe = df_chn.pivot_table(index='Brand',columns='Note',values='Total final',aggfunc=sum).round(0)
    poe['Total'] = poe.sum(axis=1).round(0)
    df_list.append(poe)
    
    for j in ['Paid','Owned','Earned']:
        df_chn_poe = df_chn[df_chn['Note']==j]
        sent_by_chn_poe = df_chn_poe.pivot_table(index=['Sentiment'], values=channel, aggfunc='sum').round(0)
        sent_by_chn_poe['Total_sent'] = sent_by_chn_poe.sum(axis=1)
        sent_by_chn_poe = sent_by_chn_poe.transpose().reindex(channel, columns = sentiment).fillna(0)
        sent_by_chn_poe['Total'] = sent_by_chn_poe.sum(axis=1)
        sent_by_chn_poe.index.name = str(i) + '_' + str(j)
        df_list.append(sent_by_chn_poe)
    

    lb_chn = df_working.pivot_table(index='Labels2', columns='Channel', values='Id', aggfunc='count').reindex(columns=channel).fillna(0)
    # Calculate row sums for sorting
    row_sums = lb_chn.sum(axis=1)
    lb_chn = lb_chn.loc[row_sums.sort_values(ascending=False).index]
    # Normalize each row by its total sum and round to 3 decimal places
    lb_chn = lb_chn.div(lb_chn.sum(axis=1), axis=0).round(3).reset_index()
    lb_chn = lb_chn.drop(lb_chn[lb_chn["Labels2"] == 0].index).head(10)
    df_list.append(lb_chn)

    for k in ['Positive','Negative']:
        labels_sent = df_lb.groupby('Labels2')['Labels2',k].sum().round(0).sort_values(by=k ,ascending = False)
        df_list.append(labels_sent)
        
    for i_sent in ['Positive', 'Negative']:
        df_sentiment = df_working[df_working['Sentiment'] == i_sent]
        top_sources = (
            df_sentiment.groupby(['Title', 'Channel', 'UrlTopic'])
            .agg({'Id': 'count'})
            .sort_values(by='Id', ascending=False)
            .head(10)
            .reset_index())
        top_sources['Link'] = top_sources.apply(lambda row: '=HYPERLINK("' + row['UrlTopic'] + '","' + row['Title'][:30] + ' ...")', axis=1)
        top_sources = top_sources.drop(['Title', 'UrlTopic'], axis=1).reindex(['Link', 'Channel', 'Id'], axis=1).set_index('Link')
        top_sources.index.name = f"Top Sources ({i_sent})"
        df_list.append(top_sources)
        
    top_sources_cmt = (
    df_working.groupby(['Title', 'SiteName', 'UrlTopic'])
        .agg({'Comments': 'sum'})
        .sort_values(by='Comments', ascending=False)
        .head(10)
        .reset_index())
    top_sources_cmt['Link'] = top_sources_cmt.apply(lambda row: '=HYPERLINK("' + row['UrlTopic'] + '","' + row['Title'][:50] + ' ...")', axis=1)
    top_sources_cmt = top_sources_cmt.drop(['Title', 'UrlTopic'], axis=1).reindex(['Link', 'SiteName', 'Comments'], axis=1).set_index('Link')
    top_sources_cmt.index.name = f"Top Comments ({i})"
    df_list.append(top_sources_cmt)
    
    top_sources_shares = (
    df_working.groupby(['Title', 'SiteName', 'UrlTopic'])
        .agg({'Shares': 'sum'})
        .sort_values(by='Shares', ascending=False)
        .head(10)
        .reset_index())
    top_sources_shares['Link'] = top_sources_shares.apply(lambda row: '=HYPERLINK("' + row['UrlTopic'] + '","' + row['Title'][:50] + ' ...")', axis=1)
    top_sources_shares = top_sources_shares.drop(['Title', 'UrlTopic'], axis=1).reindex(['Link', 'SiteName', 'Shares'], axis=1).set_index('Link')
    top_sources_shares.index.name = f"Top Shares ({i})"
    df_list.append(top_sources_shares)

    
#print(df_list)  

C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_17004\1913602683.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  overall = df_chn.groupby('Brand')['MXH', 'News', 'Total final'].sum().reset_index()
C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_17004\1913602683.py:30: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  labels = df_lb.groupby(['Labels2'])['Brand','Labels2','Total final'].sum().round(0).sort_values(by='Total final',ascending = False)
C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_17004\1913602683.py:63: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  labels_sent = df_lb.groupby('Labels2')['Labels2',k].sum().round(0).sort_values(by=k ,ascending = False)
C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_17004\1913602683

In [8]:
Ashima = []
KTOP = []
Mala_Gang = []

counts = len(df_list)
df_counts = 0  # Counter for the DataFrames

for df_item in df_list:
    df_counts += 1  # Increment the counter
    
    if df_counts < (counts/3+1):
        Ashima.append(df_item)
        slide_n_dict['Ashima'] = Ashima
    
    elif df_counts < ((counts/3)*2+1):
        KTOP.append(df_item)
        slide_n_dict['KTOP'] = KTOP
        
    else:
        Mala_Gang.append(df_item)
        slide_n_dict['Mala Gang'] = Mala_Gang
        
#print(slide_n_dict)

In [9]:
import pandas as pd

def to_excel_file(file_name, dict_sheet_df):

    file_directory = "C" + ":" + "\ ".strip() + "Users" + "\ ".strip() + "Kompa-KTelt056" + "\ ".strip() + "Desktop" \
                     + "\ ".strip() + str(file_name) + ".xlsx"
    with pd.ExcelWriter(file_directory) as writer:
        for slide, dataframe in dict_sheet_df.items():

            if type(dataframe) == list:
                start_column = 0
                for each_df in dataframe:
                    each_df.to_excel(writer, sheet_name=slide, startcol=start_column)
                    start_column += len(each_df.axes[1]) + 3

            if (type(dataframe) == pd.DataFrame) or (type(dataframe) == pd.Series):
                dataframe.to_excel(writer, sheet_name=slide)

competitors

In [10]:
topic_groups = {
    'Ashima & compe': ['Ashima', 'Asanoha', 'Momo Paradise'],
    'KTOP & compe': ['KTOP', 'Dookki'],
    'Malagang & compe': ['Mala Gang', 'Mala Panda', 'Yangguofu', 'Dookki']
}

In [11]:

for group_name, topics in topic_groups.items():
    
    df_compe_list =[]
    
    sent_compe = sent[(sent[period] == period_time) & sent['Brand'].isin(topics)].copy()
    sent_compe = sent_compe.groupby('Brand')[['Total final','Positive','Neutral','Negative']].sum().reset_index()
    sent_compe.loc[:, 'NSR'] = (sent_compe['Positive']-sent_compe['Negative'])/(sent_compe['Positive']+sent_compe['Negative'])
    NSR = sent_compe[['Total final','NSR','Brand']].round(3)
    df_compe_list.append(NSR)
    df_compe_list.append(sent_compe)
    
    lb_sent_compe = lb[(lb[period] == period_time) & lb['Brand'].isin(topics)]
    total_brand = lb_sent_compe.groupby('Brand')[['Positive','Neutral','Negative']].sum().sum(axis=1)
    total_brand.name = 'total_brand'
    labels = lb_sent_compe.groupby(['Brand','Labels2'])[['Positive','Neutral','Negative']].sum().fillna(0).round(3).reset_index()
    labels = pd.merge(labels, total_brand, how='left', on='Brand')
    labels[['Positive', 'Neutral', 'Negative']] = labels[['Positive', 'Neutral', 'Negative']].div(labels['total_brand'], axis=0).round(3)
    labels['total_brand'] = labels[['Positive', 'Neutral', 'Negative']].sum(axis=1)
    labels = labels.sort_values(by= ['Brand','total_brand'], ascending = [True,False])
    for brand in topics:
        labels_brand = labels[labels['Brand'] == brand].head(5)
        labels_brand.index.name = 'Sent_Lb'
        df_compe_list.append(labels_brand)
    
    lb_per = lb_sent_compe.groupby(['Brand','Labels2'])['Total final'].sum().fillna(0).round(3).reset_index()
    total_per_brand = lb_sent_compe.groupby('Brand')['Total final'].sum().fillna(0).round(3)
    total_per_brand.name = 'total_per_brand'
    lb_per = pd.merge(lb_per, total_per_brand, how='left', on='Brand')
    lb_per['percentage'] = (lb_per['Total final'] / lb_per['total_per_brand'])
    lb_per = lb_per[['Brand','Labels2','percentage']].sort_values(by = ['Brand','percentage'], ascending = [True,False])
    
    for brand in topics:
        lb_per_brand = lb_per[lb_per['Brand'] == brand].head(10)
        lb_per_brand.index.name = '%Labels'
        df_compe_list.append(lb_per_brand)
    
    
    df_working_compe = df[df['Topic'].isin(topics)]
    date_compe = datemerge(df_working_compe.groupby(['PublishedDate','Topic']).agg({'Id': 'count'}).pivot_table(index=['PublishedDate'], values='Id', columns='Topic',aggfunc='sum'), '2024-01-01', '2024-12-31')
    df_compe_list.append(date_compe)
    
    df_fp = df[(df['Topic'].isin(topics)) & (df['Channel']=='Fanpage') & (df['Type']=='fbPageTopic')]
    fp_inter = df_fp.pivot_table(index='Topic',values=['Comments','Shares','Likes'], aggfunc = sum)
    fp_inter['post'] = df_fp.groupby('Topic')['Id'].count()
    df_compe_list.append(fp_inter)
    
    slide_n_dict[group_name] = df_compe_list

In [12]:
to_excel_file('HotpotOutput', slide_n_dict)